Setup: Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


Get Data

In [2]:
data = pd.read_excel('dataset/Package Pricing at Mission Hospital - Data Supplement.xlsx', sheet_name='MH-Raw Data')

Data Cleaning

In [3]:
data['MARITAL STATUS'] = data['MARITAL STATUS'].str.capitalize()
data['PAST MEDICAL HISTORY CODE'] = data['PAST MEDICAL HISTORY CODE'].str.capitalize()
data['MODE OF ARRIVAL'] = data['MODE OF ARRIVAL'].str.capitalize()
data['STATE AT THE TIME OF ARRIVAL'] = data['STATE AT THE TIME OF ARRIVAL'].str.capitalize()
data['TYPE OF ADMSN'] = data['TYPE OF ADMSN'].str.capitalize()

data['AGE'] = data['AGE'].astype('int64')


Data Prep

In [4]:
# Age group categories
data['AGE_GROUP'] = '50+'

age_conditions = [
    (data['AGE'] <= 10),
    ((data['AGE'] > 10) & (data['AGE'] <= 25)),
    ((data['AGE'] > 25) & (data['AGE'] <= 50))
]

age_groups = ['<10', '11-25', '26-50']

data['AGE_GROUP'] = np.select(age_conditions, age_groups, default='50+')

# Hemoglobin Categories
data['HB_LEVEL'] = 'ABNORMAL'
data['HB_LEVEL'] = np.where(
    ((data['GENDER'] == 'F') & (data['HB'] >= 12.5) & (data['HB'] <= 15.5)) |
    ((data['GENDER'] == 'M') & (data['HB'] >= 13) & (data['HB'] <= 17.5)), 
    'NORMAL', 'ABNORMAL')

# BP Categories
bp_conditions = [
    (data['BP -HIGH'] < 120) & (data['BP-LOW'] < 80),
    (data['BP -HIGH'] > 119) & (data['BP -HIGH'] < 130) & (data['BP-LOW'] < 80),
    ((data['BP -HIGH'] > 129) & (data['BP -HIGH'] < 140)) | ((data['BP-LOW'] > 79) & (data['BP-LOW'] < 90)),
    (data['BP -HIGH'] >= 140) | (data['BP-LOW'] >= 90),
    (data['BP -HIGH'] > 180) | (data['BP-LOW'] > 120)
]

bp_groups = ['NORMAL','ELEVATED', 'HYPERTENSION STAGE 1', 'HYPERTENSION STAGE 2', 'HYPERTENSIVE CRISIS']

data['BP_RANGE'] = np.select(bp_conditions, bp_groups, default='HYPERTENSIVE CRISIS')

# Urea Categories
data['UREA_CAT'] = np.where(
    ((data['GENDER'] == 'F') & (data['UREA'] >= 6) & (data['UREA'] < 21)) |
    ((data['GENDER'] == 'M') & (data['UREA'] >= 7) & (data['UREA'] <= 20)), 
    'NORMAL', 'ABNORMAL'
)


# BMI Categories
# Calculate BMI
data['BMI_VALUE'] = data['BODY WEIGHT'] / (data['BODY HEIGHT']/100) ** 2

bmi_conditions = [
    (data['BMI_VALUE'] <= 18.5),
    (data['BMI_VALUE'] > 18.5) & (data['BMI_VALUE'] < 25),
    (data['BMI_VALUE'] >= 25) & (data['BMI_VALUE'] < 30),
    (data['BMI_VALUE'] >= 30)
]

bmi_groups = ['UNDERWEIGHT', 'NORMAL', 'OVERWEIGHT', 'OBESE']

data['BMI_CAT'] = np.select(bmi_conditions, bmi_groups, default='OBESE')

# Creatinine Categories
data['CREATININE_CAT'] = np.where(
    ((data['AGE'] <= 3) & (data['CREATININE'] >= 0.3) & (data['CREATININE'] <= 0.7)) |
    ((data['AGE'] > 3) & (data['AGE'] <= 18) & (data['CREATININE'] >= 0.5) & (data['CREATININE'] <= 1.0)) |
    ((data['AGE'] > 18) & (data['GENDER'] == 'F') & (data['CREATININE'] >= 0.6) & (data['CREATININE'] <= 1.1)) |
    ((data['AGE'] > 18) & (data['GENDER'] == 'M') & (data['CREATININE'] >= 0.9) & (data['CREATININE'] <= 1.3)),
    'NORMAL',
    'ABNORMAL'
)

data['LOG_TOTAL_COST_TO_HOSPITAL'] = np.log(data['TOTAL COST TO HOSPITAL '])